# Connection

In [1]:
import sqlite3

path = 'dpc.db'

# Flatten

In [2]:
__author__ = "Arezou Soltani Panah"
__date__ = "April 25, 2019"


"""Flattens out Tweet dictionaries so relevant JSON is in a top-level dictionary. And the store the flattened Tweets in csv files."""

import sys,requests, datetime, decimal
import ijson, csv
from collections import OrderedDict


def flattenjson( b, delim ):
    val = {}
    for i in b.keys():
        if isinstance( b[i], dict ):
            get = flattenjson( b[i], delim )
            for j in get.keys():
                val[ i + delim + j ] = get[j]
        else:
            val[i] = b[i]
    return val


def flatten_file(path):
    with open(path, 'rb') as input_file:
        # The 'results parameter may to be replaced based on the JSON attributes
        objects = ijson.items(input_file, 'results.item')

        flat_data_list = list(map(lambda x: flattenjson(x, "."), objects))
        print(len(flat_data_list))
        cleaned_flat_data_list = flat_data_list[::-1]
        global total
        total += len(cleaned_flat_data_list)
        
        # validate location
        for tweet in cleaned_flat_data_list:
            if not tweet['user.location']:
                tweet['user.location'] = 'Australia'

        # remove undesired columns
        columns = [x for row in cleaned_flat_data_list for x in row.keys()]
        columns = list(OrderedDict.fromkeys(columns))

        with open("output/"+path.split('/')[1].split('.')[0] + ".csv", 'w', encoding='utf8', errors='replace') as csvfile:
            csv_w = csv.writer(csvfile)
            csv_w.writerow(columns)
            for i_r in cleaned_flat_data_list:
                csv_w.writerow(map(lambda x: i_r.get(x, ""), columns))

            csvfile.close()

In [3]:
import os

filenames = [filename.split('.')[0] for filename in os.listdir('input') if 'DS' not in filename]
filenames.sort()
filenames

['20140101',
 '20140108',
 '20140115',
 '20140122',
 '20140129',
 '20140205',
 '20140212',
 '20140219',
 '20140226',
 '20140305',
 '20140312',
 '20140319',
 '20140326',
 '20140402',
 '20140409',
 '20140416',
 '20140423',
 '20140430',
 '20140507',
 '20140514',
 '20140521',
 '20140528',
 '20140604',
 '20140611',
 '20140618',
 '20140625',
 '20140702',
 '20140709',
 '20140716',
 '20140723',
 '20140730',
 '20140806',
 '20140813',
 '20140820',
 '20140827',
 '20140903',
 '20140910',
 '20140917',
 '20140924',
 '20141001',
 '20141008',
 '20141015',
 '20141022',
 '20141029',
 '20141105',
 '20141112',
 '20141119',
 '20141126',
 '20141203',
 '20141210',
 '20141217',
 '20141224',
 '20150101',
 '20150108',
 '20150115',
 '20150122',
 '20150129',
 '20150205',
 '20150212',
 '20150219',
 '20150226',
 '20150305',
 '20150312',
 '20150319',
 '20150326',
 '20150402',
 '20150409',
 '20150416',
 '20150423',
 '20150430',
 '20150507',
 '20150514',
 '20150521',
 '20150528',
 '20150604',
 '20150611',
 '20150618',

In [4]:
total = 0

for filename in filenames:
    if 'count' not in filename:
        print(filename, end=' ')
        flatten_file('input/'+filename+'.json')
        
total

20140101 2
20140108 62
20140115 10
20140122 13
20140129 10
20140205 52
20140212 37
20140219 21
20140226 79
20140305 49
20140312 17
20140319 35
20140326 24
20140402 31
20140409 20
20140416 87
20140423 78
20140430 102
20140507 118
20140514 106
20140521 38
20140528 72
20140604 66
20140611 20
20140618 79
20140625 85
20140702 47
20140709 111
20140716 48
20140723 30
20140730 53
20140806 137
20140813 43
20140820 57
20140827 65
20140903 68
20140910 102
20140917 114
20140924 134
20141001 75
20141008 108
20141015 84
20141022 86
20141029 127
20141105 210
20141112 72
20141119 293
20141126 144
20141203 69
20141210 49
20141217 112
20141224 60
20150101 80
20150108 67
20150115 298
20150122 303
20150129 227
20150205 336
20150212 222
20150219 500
20150226 51
20150305 1
20150312 0
20150319 3
20150326 2
20150402 1
20150409 1
20150416 62
20150423 254
20150430 500
20150507 500
20150514 499
20150521 499
20150528 500
20150604 500
20150611 499
20150618 500
20150625 500
20150702 500
20150709 500
20150716 500
20

107840

# Import to DB

In [5]:
import pandas
conn = sqlite3.connect(path)

for filename in filenames:
    if 'count' not in filename:
        if len(open('output/'+filename+'.csv', 'r').readlines()) > 1:
            print(filename, end=' ')
            df = pandas.read_csv('output/'+filename+'.csv')
            df.to_sql('tweets', conn, if_exists='append', index=False)

20140101 20140108 20140115 20140122 20140129 20140205 20140212 20140219 20140226 20140305 20140312 20140319 20140326 20140402 20140409 20140416 20140423 20140430 20140507 20140514 20140521 20140528 20140604 20140611 20140618 20140625 20140702 20140709 20140716 20140723 20140730 20140806 20140813 20140820 20140827 20140903 20140910 20140917 20140924 20141001 20141008 20141015 20141022 20141029 20141105 20141112 20141119 20141126 20141203 20141210 20141217 20141224 20150101 20150108 20150115 20150122 20150129 20150205 20150212 20150219 20150226 20150305 20150319 20150326 20150402 20150409 20150416 20150423 20150430 20150507 20150514 20150521 20150528 20150604 20150611 20150618 20150625 20150702 20150709 20150716 20150723 20150730 20150806 20150813 20150820 20150827 20150903 20150910 20150917 20150924 20151001 20151008 20151015 20151022 20151029 20151105 20151112 20151119 20151126 20151203 20151210 20151217 20151224 20160101 20160108 20160115 20160122 20160129 20160205 20160212 20160219 2

# Count

In [15]:
import json

dates = []
counts = []
start_dates = []
end_dates = []
total_counts = []

for filename in filenames:
    if 'count' in filename:
        for line in open('input/'+filename+'.json').readlines():
            if line.strip() != '':
                obj = json.loads(line)
                for o in obj['results']:
                    dates.append(o['timePeriod'][:8])
                    counts.append(o['count'])
                total_counts.append(obj['totalCount'])
                start_dates.append(obj['requestParameters']['fromDate'][:8])
                end_dates.append(str(int(obj['requestParameters']['toDate'][:8])-1))
                
len(total_counts)

276

In [16]:
available_tweet_counts = []
total = 0
for filename in filenames:
    if 'count' not in filename:
        with open('input/'+filename+'.json') as f:
            obj = json.load(f)
            if 'results' in obj:
                available_tweet_counts.append(len(obj['results']))
                total += len(obj['results'])
            else:
                available_tweet_counts.append(0)
                
total

107840

In [17]:
df = pandas.DataFrame.from_dict({'date': dates, 'count': counts})
df.to_sql('count_tweets_daily', conn, if_exists='replace', index=False)

In [18]:
df = pandas.DataFrame.from_dict({'start_date': start_dates, 'end_date': end_dates, 'available_tweet_count': available_tweet_counts, 'total_count': total_counts})
df.to_sql('count_tweets_weekly', conn, if_exists='replace', index=False)